# Data Wrangling

This JSON data is in a strange format.  The columns appear in an array at the top, and then records appear in an array in a separate field.  We need to create a dataframe with this data.

In [2]:
import os
import pandas as pd
import json

f = open("data/2020-7.json")
data = json.load(f)
f.close()

df = pd.DataFrame(columns=data["columns"])


In [3]:
for fname in os.listdir("data"):
    f = open("data/" + fname)
    data = json.load(f)
    f.close()
    df = df.append(data["records"], ignore_index=True)

df.shape

(4077, 20)

In [48]:
df.to_csv("fulldata/data.csv")

BlockingIOError: [Errno 35] Resource temporarily unavailable: 'fulldata/data.csv'

In [5]:
df.subdivisiondescr.fillna("NONE",inplace=True)

In [6]:
subdivisions = df.subdivisiondescr.unique()

In [7]:
subdivisions.sort()

In [8]:
subdivisions

array(['435 EAST DRAKE CONDO AMD', 'ABERDEEN AT WATERGLEN',
       'ABERDEEN AT WATERGLEN CONDO SUPP. NO. 1',
       'ABERDEEN AT WATERGLEN CONDO, SUPP NO.9,',
       'ABERDEEN AT WATERGLEN SUPP 11 BLDG J CONDOS FTC (20050010812)',
       'ABERDEEN AT WATERGLEN, SUPP 6 (BLDG G), FTC (20040010844)',
       'ABERDEEN AT WATERGLEN, SUPP NO. 7, FTC (20040035245)',
       'ADER ESTATES PUD', 'ADRIEL HILLS', 'ADRIEL HILLS OVERALL',
       'ADRIEL HILLS TWO', 'ADRIEL VILLAGE', 'ALTA VISTA',
       'ALVIN L MILLER', 'ANDERSON PLACE',
       'APPALOOSA AT LORY ANN ESTATES CONDOS', 'APPLERIDGE PUD',
       'APPLERIDGE TOWNHOMES', 'APPLEWOOD ESTATES',
       'ARAPAHOE FARM TOWNHOMES PUD', 'ARAPAHOE HILLS',
       'ARAPAHOE PINES CONDOS', 'ARROWHEAD II CONDOS BLDG 3 AMNDED MAP',
       'ARTALICE', 'ASHFORD COMMONS PUD', 'ASPEN GROVE CONDOS',
       'ASPEN VILLAGE CONDOS',
       'AUBURN ESTATES CONSERVATION DEVELOPMENT 03-S2143 (20070021442)',
       'AURORA HILLS', 'AUTUMN CREEK CONS DEV, 2ND FLG

In [12]:
subdivisions

array(['435 EAST DRAKE CONDO AMD', 'ABERDEEN AT WATERGLEN',
       'ABERDEEN AT WATERGLEN CONDO SUPP. NO. 1',
       'ABERDEEN AT WATERGLEN CONDO, SUPP NO.9,',
       'ABERDEEN AT WATERGLEN SUPP 11 BLDG J CONDOS FTC (20050010812)',
       'ABERDEEN AT WATERGLEN, SUPP 6 (BLDG G), FTC (20040010844)',
       'ABERDEEN AT WATERGLEN, SUPP NO. 7, FTC (20040035245)',
       'ADER ESTATES PUD', 'ADRIEL HILLS', 'ADRIEL HILLS OVERALL',
       'ADRIEL HILLS TWO', 'ADRIEL VILLAGE', 'ALTA VISTA',
       'ALVIN L MILLER', 'ANDERSON PLACE',
       'APPALOOSA AT LORY ANN ESTATES CONDOS', 'APPLERIDGE PUD',
       'APPLERIDGE TOWNHOMES', 'APPLEWOOD ESTATES',
       'ARAPAHOE FARM TOWNHOMES PUD', 'ARAPAHOE HILLS',
       'ARAPAHOE PINES CONDOS', 'ARROWHEAD II CONDOS BLDG 3 AMNDED MAP',
       'ARTALICE', 'ASHFORD COMMONS PUD', 'ASPEN GROVE CONDOS',
       'ASPEN VILLAGE CONDOS',
       'AUBURN ESTATES CONSERVATION DEVELOPMENT 03-S2143 (20070021442)',
       'AURORA HILLS', 'AUTUMN CREEK CONS DEV, 2ND FLG

In [22]:
df.subdivisiondescr.str.replace(r'^(BIG HORN VILLAGE)', 'BIG HORN VILLAGE')


<ipython-input-22-5f30b13da221>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.subdivisiondescr.str.replace(r'^(BIG HORN VILLAGE)', 'BIG HORN VILLAGE')


0                               NONE
1                    MILLER BROTHERS
2                         S9 T07 R69
3                           WOODWEST
4                   WEST LAKE STREET
                    ...             
4072                  SHER-ELL ACRES
4073          MOUNTAIN RANGE SHADOWS
4074    MALLARDS AT THE LANDINGS PUD
4075                  ADRIEL VILLAGE
4076          BUCKING HORSE FILING 1
Name: subdivisiondescr, Length: 4077, dtype: object

Now we have a lot of data cleanup to do, as we do want to have the neighborhood as a feature.  Many of the neighborhoods are broken up, and we'd like to combine them together.

In many cases, in a condominium subdivision, each building counts as a separate subdivision.  For my work, all condos in a group count as one "neighborhood."

In [34]:
df.CleanSubdivision = df.subdivisiondescr
df.loc[df.subdivisiondescr.str.startswith("BIG HORN VILLAGE"), "CleanSubdivision"] = 'BIG HORN VILLAGE'
df.loc[df.subdivisiondescr.str.startswith("ABERDEEN"), "CleanSubdivision"] = 'ABERDEEN'
df.loc[df.subdivisiondescr.str.startswith("ADRIEL HILLS"), "CleanSubdivision"] = 'ADRIEL HILLS'
df.loc[df.subdivisiondescr.str.startswith("APPLERIDGE"), "CleanSubdivision"] = 'APPLERIDGE'
df.loc[df.subdivisiondescr.str.startswith("BELLA VIRA"), "CleanSubdivision"] = 'BELLA VIRA'
df.loc[df.subdivisiondescr.str.startswith("BOARDWALK OFFICE"), "CleanSubdivision"] = 'BOARDWALK OFFICE'
df.loc[df.subdivisiondescr.str.startswith("BROOKSIDE AT WILLOW SPRINGS"), "CleanSubdivision"] = 'BROOKSIDE AT WILLOW SPRINGS'
df.loc[df.subdivisiondescr.str.startswith("BUCKING HORSE"), "CleanSubdivision"] = 'BUCKING HORSE'
df.loc[df.subdivisiondescr.str.startswith("CLYDESDALE PARK"), "CleanSubdivision"] = 'CLYDESDALE PARK'
df.loc[df.subdivisiondescr.str.startswith("COUNTRY CLUB ESTATES"), "CleanSubdivision"] = 'COUNTRY CLUB ESTATES'
df.loc[df.subdivisiondescr.str.startswith("CROSSING AT FOSSIL LAKE CONDOS"), "CleanSubdivision"] = 'CROSSING AT FOSSIL LAKE CONDOS'
df.loc[df.subdivisiondescr.str.startswith("EAGLE CLIFFS"), "CleanSubdivision"] = 'EAGLE CLIFFS'
df.loc[df.subdivisiondescr.str.startswith("EAGLE RANCH ESTATES"), "CleanSubdivision"] = 'EAGLE RANCH ESTATES'
df.loc[df.subdivisiondescr.str.startswith("EAST VILLAGE AT RIGDEN FARM"), "CleanSubdivision"] = 'EAST VILLAGE AT RIGDEN FARM'
df.loc[df.subdivisiondescr.str.startswith("EDORA ACRES"), "CleanSubdivision"] = 'EDORA ACRES'
df.loc[df.subdivisiondescr.str.startswith("ELLIOTT MILLER"), "CleanSubdivision"] = 'ELLIOTT MILLER'
df.loc[df.subdivisiondescr.str.startswith("ENGLISH RANCH"), "CleanSubdivision"] = 'ENGLISH RANCH'
df.loc[df.subdivisiondescr.str.startswith("FLATS AT RIGDEN FARM"), "CleanSubdivision"] = 'FLATS AT RIGDEN FARM'
df.loc[df.subdivisiondescr.str.startswith("FOSSIL CREEK CONDOMINIUMS"), "CleanSubdivision"] = 'FOSSIL CREEK CONDOMINIUMS'
df.loc[df.subdivisiondescr.str.startswith("FOSSIL LAKE PUD"), "CleanSubdivision"] = 'FOSSIL LAKE PUD'
df.loc[df.subdivisiondescr.str.startswith("FOX MEADOWS"), "CleanSubdivision"] = 'FOX MEADOWS'
df.loc[df.subdivisiondescr.str.startswith("HARMONY RIDGE"), "CleanSubdivision"] = 'HARMONY RIDGE'
df.loc[df.subdivisiondescr.str.startswith("HARVEST PARK"), "CleanSubdivision"] = 'HARVEST PARK'
df.loc[df.subdivisiondescr.str.startswith("HEARTHFIRE PUD"), "CleanSubdivision"] = 'HEARTHFIRE PUD'
df.loc[df.subdivisiondescr.str.startswith("HIGHLAND HILLS"), "CleanSubdivision"] = 'HIGHLAND HILLS PUD'
df.loc[df.subdivisiondescr.str.startswith("HORSETOOTH LAKE"), "CleanSubdivision"] = 'HORSETOOTH LAKE ESTATES'
df.loc[df.subdivisiondescr.str.startswith("KENSINGTON SOUTH"), "CleanSubdivision"] = 'KENSINGTON SOUTH'
df.loc[df.subdivisiondescr.str.startswith("LAGRANGE CONDO"), "CleanSubdivision"] = 'LAGRANGE CONDOS'
df.loc[df.subdivisiondescr.str.startswith("LINDEN PARK"), "CleanSubdivision"] = 'LINDEN PARK'
df.loc[df.subdivisiondescr.str.startswith("LODGE AT MIRAMONT"), "CleanSubdivision"] = 'LODGE AT MIRAMONT'
df.loc[df.subdivisiondescr.str.startswith("MACKENZIE PLACE OAKRIDGE CONDOS"), "CleanSubdivision"] = 'MACKENZIE PLACE OAKRIDGE CONDOS'
df.loc[df.subdivisiondescr.str.startswith("MAPLE HILL"), "CleanSubdivision"] = 'MAPLE HILL'
df.loc[df.subdivisiondescr.str.startswith("MORNINGSIDE VILLAGE"), "CleanSubdivision"] = 'MORNINGSIDE VILLAGE'
df.loc[df.subdivisiondescr.str.startswith("MOUNTAIN RIDGE FARM"), "CleanSubdivision"] = 'MOUNTAIN RIDGE FARM'
df.loc[df.subdivisiondescr.str.startswith("NORTH FLATS"), "CleanSubdivision"] = 'NORTH FLATS CONDOS'
df.loc[df.subdivisiondescr.str.startswith("OLD TOWN NORTH"), "CleanSubdivision"] = 'OLD TOWN NORTH'
df.loc[df.subdivisiondescr.str.startswith("PARKSIDE EAST AT"), "CleanSubdivision"] = 'PARKSIDE EAST CONDOS'
df.loc[df.subdivisiondescr.str.startswith("PARKSIDE EAST CONDO"), "CleanSubdivision"] = 'PARKSIDE EAST CONDOS'
df.loc[df.subdivisiondescr.str.startswith("PARKSIDE WEST AT"), "CleanSubdivision"] = 'PARKSIDE WEST CONDOS'
df.loc[df.subdivisiondescr.str.startswith("PROVINCETOWNE"), "CleanSubdivision"] = 'PROVINCETOWNE'
df.loc[df.subdivisiondescr.str.startswith("QUAIL HOLLOW"), "CleanSubdivision"] = 'QUAIL HOLLOW'
df.loc[df.subdivisiondescr.str.startswith("REGISTRY RIDGE"), "CleanSubdivision"] = 'REGISTRY RIDGE'
df.loc[df.subdivisiondescr.str.startswith("RIDGEWOOD HILLS PUD"), "CleanSubdivision"] = 'RIDGEWOOD HILLS PUD'
df.loc[df.subdivisiondescr.str.startswith("RIGDEN FARM"), "CleanSubdivision"] = 'RIGDEN FARM'
df.loc[df.subdivisiondescr.str.startswith("RIVER MODERN SUB"), "CleanSubdivision"] = 'RIVER MODERN SUB'
df.loc[df.subdivisiondescr.str.startswith("RIVER ROCK COMMON"), "CleanSubdivision"] = 'RIVER ROCK COMMON CONDOS'
df.loc[df.subdivisiondescr.str.startswith("SADDLE RIDGE"), "CleanSubdivision"] = 'SADDLE RIDGE'
df.loc[df.subdivisiondescr.str.startswith("SIDEHILL CONDOS"), "CleanSubdivision"] = 'SIDEHILL CONDOS'
df.loc[df.subdivisiondescr.str.startswith("SILVER OAK"), "CleanSubdivision"] = 'SILVER OAK'
df.loc[df.subdivisiondescr.str.startswith("SOUTH GLEN CONDOS"), "CleanSubdivision"] = 'SOUTH GLEN CONDOS'
df.loc[df.subdivisiondescr.str.startswith("SOUTH GLEN PUD"), "CleanSubdivision"] = 'SOUTH GLEN PUD'
df.loc[df.subdivisiondescr.str.startswith("TIMBERS CONDO"), "CleanSubdivision"] = 'TIMBERS CONDOS'
df.loc[df.subdivisiondescr.str.startswith("TRIPLE CROWN HOMES AT OBSERVATORY VILLAGE"), "CleanSubdivision"] = 'TRIPLE CROWN HOMES AT OBSERVATORY VILLAGE'
df.loc[df.subdivisiondescr.str.startswith("VILLAGE AT FOSSIL LAKE"), "CleanSubdivision"] = 'VILLAGE AT FOSSIL LAKE CONDOS'
#df.loc[df.subdivisiondescr.str.startswith("XXXXXX"), "CleanSubdivision"] = 'XXXXXX'



In [35]:
df

,accountno,parcelnb,saledt,locationaddress,locationcity,locationzipcode,subdivisionid,subdivisiondescr,nbhd,occdescr,...,bltasid,residentialsf,bsmntsf,bsmntfinsf,garagesf,grossacres,yrblt,salep,value,CleanSubdivision
0,M0927775,9702100021,2020-07-31,1601 N COLLEGE AVE 65,FORT COLLINS,80524,None,NONE,MH2,MH In Park,...,14,672,None,None,None,.00000000,1970,14000.00,4700.00,NONE
1,R0076678,9715207002,2020-07-31,1804 CRESTMORE PL,FORT COLLINS,80521,11203,MILLER BROTHERS,19715,Single Family Residential,...,1,1080,None,None,297,.15000000,1958,370000.00,352500.00,MILLER BROTHERS
2,R0147648,9709300018,2020-07-31,2530 W MULBERRY ST,FORT COLLINS,80521,/090769,S9 T07 R69,19715,Single Family Residential,...,1,890,None,None,280,.42000000,1900,347500.00,259100.00,S9 T07 R69
3,R0135194,9726210081,2020-07-31,707 WINCHESTER DR,FORT COLLINS,80526,12302,WOODWEST,19722,Single Family Residential,...,7,1636,448,None,432,.19000000,1975,395000.00,370600.00,WOODWEST
4,R0095451,9716412002,2020-07-31,1209 BRIARWOOD RD,FORT COLLINS,80521,1219,WEST LAKE STREET,19715,Single Family Residential,...,7,1580,None,None,480,.22000000,1972,386000.00,377200.00,WEST LAKE STREET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4072,R0161179,8831407032,2021-08-02,1905 RANGEVIEW DR,FORT COLLINS,80524,0248,SHER-ELL ACRES,19836,Single Family Residential,...,7,3196,None,None,714,.39000000,1977,800000.00,628600.00,SHER-ELL ACRES
4073,R0719790,8622318044,2021-08-02,4408 FLATTOP CT,FORT COLLINS,80528,03972,MOUNTAIN RANGE SHADOWS,19613,Single Family Residential,...,7,1344,None,None,288,.11000000,1979,352000.00,260100.00,MOUNTAIN RANGE SHADOWS
4074,R1280929,9736431040,2021-08-02,4566 SEABOARD LN,FORT COLLINS,80525,1643,MALLARDS AT THE LANDINGS PUD,19601,Single Family Residential,...,1,1451,1413,1217,484,.16000000,1988,600000.00,450500.00,MALLARDS AT THE LANDINGS PUD
4075,R1285564,8831428010,2021-08-02,2016 SIMSBURY CT,FORT COLLINS,80524,0361011,ADRIEL VILLAGE,19836,Single Family Residential,...,7,2198,922,922,506,.14000000,1988,473330.00,470900.00,ADRIEL VILLAGE


In [36]:
df.to_csv("fulldata/data-post-subdivisions.csv")

In [41]:
df

,accountno,parcelnb,saledt,locationaddress,locationcity,locationzipcode,subdivisionid,subdivisiondescr,nbhd,occdescr,...,bltasid,residentialsf,bsmntsf,bsmntfinsf,garagesf,grossacres,yrblt,salep,value,CleanSubdivision
0,M0927775,9702100021,2020-07-31,1601 N COLLEGE AVE 65,FORT COLLINS,80524,None,NONE,MH2,MH In Park,...,14,672,None,None,None,.00000000,1970,14000.00,4700.00,NONE
1,R0076678,9715207002,2020-07-31,1804 CRESTMORE PL,FORT COLLINS,80521,11203,MILLER BROTHERS,19715,Single Family Residential,...,1,1080,None,None,297,.15000000,1958,370000.00,352500.00,MILLER BROTHERS
2,R0147648,9709300018,2020-07-31,2530 W MULBERRY ST,FORT COLLINS,80521,/090769,S9 T07 R69,19715,Single Family Residential,...,1,890,None,None,280,.42000000,1900,347500.00,259100.00,S9 T07 R69
3,R0135194,9726210081,2020-07-31,707 WINCHESTER DR,FORT COLLINS,80526,12302,WOODWEST,19722,Single Family Residential,...,7,1636,448,None,432,.19000000,1975,395000.00,370600.00,WOODWEST
4,R0095451,9716412002,2020-07-31,1209 BRIARWOOD RD,FORT COLLINS,80521,1219,WEST LAKE STREET,19715,Single Family Residential,...,7,1580,None,None,480,.22000000,1972,386000.00,377200.00,WEST LAKE STREET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4072,R0161179,8831407032,2021-08-02,1905 RANGEVIEW DR,FORT COLLINS,80524,0248,SHER-ELL ACRES,19836,Single Family Residential,...,7,3196,None,None,714,.39000000,1977,800000.00,628600.00,SHER-ELL ACRES
4073,R0719790,8622318044,2021-08-02,4408 FLATTOP CT,FORT COLLINS,80528,03972,MOUNTAIN RANGE SHADOWS,19613,Single Family Residential,...,7,1344,None,None,288,.11000000,1979,352000.00,260100.00,MOUNTAIN RANGE SHADOWS
4074,R1280929,9736431040,2021-08-02,4566 SEABOARD LN,FORT COLLINS,80525,1643,MALLARDS AT THE LANDINGS PUD,19601,Single Family Residential,...,1,1451,1413,1217,484,.16000000,1988,600000.00,450500.00,MALLARDS AT THE LANDINGS PUD
4075,R1285564,8831428010,2021-08-02,2016 SIMSBURY CT,FORT COLLINS,80524,0361011,ADRIEL VILLAGE,19836,Single Family Residential,...,7,2198,922,922,506,.14000000,1988,473330.00,470900.00,ADRIEL VILLAGE


In [45]:
trimdf = df.drop(columns=['accountno', 'parcelnb', 'locationcity', 'locationzipcode', 'subdivisionid', 'subdivisiondescr', 'nbhd','value'])

AttributeError: 'DataFrame' object has no attribute 'isnan'

In [50]:
trimdf.drop(columns=["locationaddress", "occdescr", "bltasoccdesc"], inplace=True)

In [59]:
trimdf["saledt"] = pd.to_datetime(trimdf["saledt"])


In [65]:
trimdf

,saledt,bltasid,residentialsf,bsmntsf,bsmntfinsf,garagesf,grossacres,yrblt,salep,CleanSubdivision,DayOfYear
0,2020-07-31,14,672,None,None,None,.00000000,1970,14000.00,NONE,213
1,2020-07-31,1,1080,None,None,297,.15000000,1958,370000.00,MILLER BROTHERS,213
2,2020-07-31,1,890,None,None,280,.42000000,1900,347500.00,S9 T07 R69,213
3,2020-07-31,7,1636,448,None,432,.19000000,1975,395000.00,WOODWEST,213
4,2020-07-31,7,1580,None,None,480,.22000000,1972,386000.00,WEST LAKE STREET,213
...,...,...,...,...,...,...,...,...,...,...,...
4072,2021-08-02,7,3196,None,None,714,.39000000,1977,800000.00,SHER-ELL ACRES,214
4073,2021-08-02,7,1344,None,None,288,.11000000,1979,352000.00,MOUNTAIN RANGE SHADOWS,214
4074,2021-08-02,1,1451,1413,1217,484,.16000000,1988,600000.00,MALLARDS AT THE LANDINGS PUD,214
4075,2021-08-02,7,2198,922,922,506,.14000000,1988,473330.00,ADRIEL VILLAGE,214


In [64]:

trimdf.DayOfYear = trimdf.saledt.dt.day_of_year

In [68]:
trimdf = trimdf.drop(columns=["saledt"])

In [69]:
trimdf

,bltasid,residentialsf,bsmntsf,bsmntfinsf,garagesf,grossacres,yrblt,salep,CleanSubdivision,DayOfYear
0,14,672,None,None,None,.00000000,1970,14000.00,NONE,213
1,1,1080,None,None,297,.15000000,1958,370000.00,MILLER BROTHERS,213
2,1,890,None,None,280,.42000000,1900,347500.00,S9 T07 R69,213
3,7,1636,448,None,432,.19000000,1975,395000.00,WOODWEST,213
4,7,1580,None,None,480,.22000000,1972,386000.00,WEST LAKE STREET,213
...,...,...,...,...,...,...,...,...,...,...
4072,7,3196,None,None,714,.39000000,1977,800000.00,SHER-ELL ACRES,214
4073,7,1344,None,None,288,.11000000,1979,352000.00,MOUNTAIN RANGE SHADOWS,214
4074,1,1451,1413,1217,484,.16000000,1988,600000.00,MALLARDS AT THE LANDINGS PUD,214
4075,7,2198,922,922,506,.14000000,1988,473330.00,ADRIEL VILLAGE,214


In [73]:
trimdf.bltasid = trimdf.bltasid.astype('int64')

In [76]:
trimdf.residentialsf = trimdf.residentialsf.astype('int64')